In [55]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [56]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [57]:
import findspark
findspark.init()

In [58]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
import numpy as np
from pyspark.sql.functions import udf, col, radians, sin, cos, sqrt, asin, sum as _sum
from pyspark.sql import functions as func
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, radians, sin, cos, sqrt, asin


In [59]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("LR1")\
        .getOrCreate()

In [60]:
trips_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trips.csv")

trips_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [61]:
stations_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("stations.csv")

stations_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



# 1 Найти велосипед с максимальным временем пробега

In [62]:
bike_max_trip = (
    trips_data
    .groupBy("bike_id")
    .agg({"duration": "sum"})
    .withColumnRenamed("sum(duration)", "total_duration")
    .sort("total_duration", ascending=False)
    .first()
)

print(f' Велосипед с id: {bike_max_trip["bike_id"]} проехал максимальное время равное: {bike_max_trip["total_duration"]}')

 Велосипед с id: 535 проехал максимальное время равное: 18611693


# 2 Найти наибольшее геодезическое расстояние между станциями.

In [63]:
stations_rad = stations_data.withColumn("lat_rad", radians(col("lat"))) \
                          .withColumn("long_rad", radians(col("long")))

# Получаем уникальные пары станций
pairs = stations_rad.alias("a") \
    .crossJoin(stations_rad.alias("b")) \
    .filter(col("a.id") < col("b.id"))

# Радиус Земли в километрах
R = 6371

# Вычисляем разности по широте и долготе
pairs = pairs.withColumn("dlat", col("b.lat_rad") - col("a.lat_rad")) \
             .withColumn("dlon", col("b.long_rad") - col("a.long_rad"))

# Вычисляем часть формулы гаверсинуса
pairs = pairs.withColumn("a_hav", sin(col("dlat") / 2) ** 2 +
                                    cos(col("a.lat_rad")) * cos(col("b.lat_rad")) * sin(col("dlon") / 2) ** 2)

# Вычисляем центральный угол
pairs = pairs.withColumn("c", 2 * asin(sqrt(col("a_hav"))))

# Вычисляем расстояние
pairs = pairs.withColumn("distance", col("c") * R)

# Находим максимальное расстояние
max_distance = pairs.agg({"distance": "max"}).collect()[0][0]

print(f"Максимальное геодезическое расстояние между станциями: {max_distance: .2f} км")

Максимальное геодезическое расстояние между станциями:  69.92 км


# 3 Найти путь велосипеда с максимальным временем пробега через станции

In [64]:
longest_bike_trip = (trips_data
                     .select('start_station_name', 'end_station_name', 'duration')
                     .orderBy(
                         col('duration')
                         .desc()
                      )
                     .first()
)

print(f'Велосипет с максимальным временем пробега = {longest_bike_trip["duration"]} '\
      f'начал движение со станции {longest_bike_trip["start_station_name"]} ' \
      f'и закончил на станции {longest_bike_trip["end_station_name"]}')

Велосипет с максимальным временем пробега = 17270400 начал движение со станции South Van Ness at Market и закончил на станции 2nd at Folsom


# 4 Найти количество велосипедов в системе

In [65]:
print(f'Количество велосипедов в системе: {trips_data.select("bike_id").distinct().count()}')

Количество велосипедов в системе: 700


# 5 Найти пользователей потративших на поездки более 3 часов

In [66]:
# 3 часа в секундах
three_hours_sec = 3 * 3600

# Группируем по zip_code и суммируем длительность поездок
users_total_duration = trips_data.groupBy("zip_code") \
                               .agg(_sum("duration").alias("total_duration"))

users_more_than_3 = users_total_duration.filter(col("total_duration") > three_hours_sec)

users_more_than_3.show()

+--------+--------------+
|zip_code|total_duration|
+--------+--------------+
|   94102|      19128021|
|   95134|        728023|
|   84606|         95145|
|   80305|        180906|
|   60070|         28919|
|   95519|         30303|
|   43085|         11670|
|   91910|         50488|
|   77339|         13713|
|   48063|         13755|
|   85022|         12682|
|    1090|         20391|
|    2136|         16010|
|   11722|         24331|
|   95138|        155295|
|   94610|       3630628|
|   94404|       3589350|
|   80301|        152189|
|   91326|         65885|
|   90742|         10965|
+--------+--------------+
only showing top 20 rows

